In [35]:
from langchain_openai.llms import OpenAI

llm = OpenAI()


In [36]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário
{pergunta}
""")

In [37]:
print(prompt_template.format(pergunta="O que é um SaaS?"))


Responda a seguinte pergunta do usuário
O que é um SaaS?



In [38]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""")

In [39]:
print(prompt_template.format(pergunta="O que é um SaaS?", n_palavras=15))


Responda a seguinte pergunta do usuário em até 15 palavras:
O que é um SaaS?



In [40]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""", partial_variables={"n_palavras": 10}
)

In [41]:
print(prompt_template.format(pergunta="O que é LangChain?"))


Responda a seguinte pergunta do usuário em até 10 palavras:
O que é LangChain?



### Utilizando Múltiplos Prompts

In [42]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_palavras} palavras:
""")

template_line_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_linhas} linhas:

""")

template_idiom = PromptTemplate.from_template("""
Retorne a resposta em {idiom}.

""")

template_final = (template_word_count + template_line_count + template_idiom +
                  "Responda a pergunta seguindo as instruções {pergunta}")

In [43]:
print(template_final)

input_variables=['idiom', 'n_linhas', 'n_palavras', 'pergunta'] template='\nResponda a pergunta em até {n_palavras} palavras:\n\nResponda a pergunta em até {n_linhas} linhas:\n\n\nRetorne a resposta em {idiom}.\n\nResponda a pergunta seguindo as instruções {pergunta}'


In [44]:
prompt_final = template_final.format(n_palavras=15, n_linhas = 1, idiom="inglês", pergunta="O que é o Sol?")
llm.invoke(prompt_final)

'\n\nThe Sun is the star at the center of our solar system.'

In [45]:
print(prompt_final)


Responda a pergunta em até 15 palavras:

Responda a pergunta em até 1 linhas:


Retorne a resposta em inglês.

Responda a pergunta seguindo as instruções O que é o Sol?


### Templates para Chat

In [46]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é a minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem é você?")

[HumanMessage(content='Essa é a minha dúvida: Quem é você?')]

In [47]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente irônico e se chama {nome_assistente}"),
        ("human", "Olá como vai?"),
        ("ai", "Estou bem, como posso lhe ajudar?"),
        ("human", "{pergunta}")
    ]
)

In [48]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente irônico e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Estou bem, como posso lhe ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [49]:
chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?")

[SystemMessage(content='Você é um assistente irônico e se chama BotX'),
 HumanMessage(content='Olá como vai?'),
 AIMessage(content='Estou bem, como posso lhe ajudar?'),
 HumanMessage(content='Qual seu nome?')]

In [50]:
from langchain_openai.chat_models import ChatOpenAI
chat=ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?"))

AIMessage(content='Meu nome é BotX. Como posso lhe ajudar hoje?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b196ca5a-123e-4802-aa45-08c487c60f59-0')

### Few Shot Prompting

In [51]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [52]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

'Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n'

In [53]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [54]:
llm.invoke(prompt.format(input="Quais são os métodos que uma classe recebe?"))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: O que é uma classe? \nResposta intermediária: Uma classe é um conceito fundamental da programação orientada a objetos que representa um conjunto de objetos com características e comportamentos semelhantes. \nPergunta de acompanhamento: Quais são os métodos em uma classe? \nResposta intermediária: Os métodos em uma classe são funções ou procedimentos que podem ser chamados para executar ações ou retornar informações sobre objetos dessa classe. \nEntão a resposta final é: Os métodos em uma classe podem variar, mas geralmente incluem funções de inicialização, getters e setters para as propriedades da classe, e outros métodos específicos para o comportamento desejado.'

In [55]:
llm.invoke(prompt.format(input="como Uma classe recebe métodos através da herança de outras classes ou da implementação de interfaces, EXPLIQUE"))

'\nSão necessárias perguntas de acompanhamento aqui: Não. \nResposta final: A herança e a implementação de interfaces permitem que uma classe receba métodos de outras classes ou interfaces, permitindo que ela utilize esses métodos em seu próprio código. A herança ocorre quando uma classe é criada a partir de outra classe, e a classe filha herda todos os métodos e atributos da classe pai. Já a implementação de interfaces ocorre quando uma classe implementa uma interface, que contém métodos que devem ser implementados pela classe. Dessa forma, a classe que implementa a interface também recebe os métodos definidos na interface. '

### Few Shot Prompting em Chat

In [56]:
from langchain_openai.chat_models import ChatOpenAI
chat=ChatOpenAI()


In [57]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate


example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
    ("ai", "{resposta}")]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Qual é a maior montanha do mundo, o Monte Everest ou o K2?'), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n')]


In [58]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="como Uma classe recebe métodos através da herança de outras classes ou da implementação de interfaces, EXPLIQUE")  

In [59]:
chat.invoke(prompt)

AIMessage(content='Quando uma classe herda de outra classe ou implementa uma interface em programação orientada a objetos, ela pode receber métodos dessas classes ou interfaces.\n\nNo caso de herança, uma classe filha pode herdar métodos da classe pai. Isso significa que a classe filha terá acesso aos métodos públicos e protegidos da classe pai, podendo utilizá-los diretamente. Essa herança permite reutilizar código e estabelecer uma relação de especialização entre as classes.\n\nJá no caso da implementação de interfaces, uma classe que implementa uma interface deve fornecer a implementação dos métodos definidos nessa interface. Isso garante que a classe terá os métodos especificados pela interface, e permite que objetos dessa classe sejam tratados de forma polimórfica, ou seja, como instâncias da interface.\n\nEm resumo, quando uma classe herda de outra classe ou implementa uma interface, ela recebe os métodos dessas classes ou interfaces, podendo utilizá-los conforme necessário em su